In [ ]:
# # important paackages
# !pip install pypdf
# ! pip install docx2txt
# ! pip install langchain-community
# ! pip install sentence-transformers
# !pip install langchain_text_splitters
# !pip install langchain_core
# ! pip install chromadb
# ! pip install langchain_openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.2/437.2 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:

In [8]:
import numpy as np
import os
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter
from langchain_text_splitters.character import CharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

#generation step
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [9]:
# load and preprocess text
loader = Docx2txtLoader("/content/Introduction_to_Data_and_Data_Science_2.docx")
pages = loader.load()

# split by markdown
md_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on= [("#", "Course Title"),
                          ("##", "Lecture Title")]
)

pages_md_split = md_splitter.split_text(pages[0].page_content)

# remove \\n
for i in range(len(pages_md_split)):
  pages_md_split[i].page_content  = " ".join(pages_md_split[i].page_content.split())


char_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=500,
    chunk_overlap=50
)

pages_char_split = char_splitter.split_documents(pages_md_split)


In [10]:
# Embeddings

In [11]:
embedding_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2") # 784 -1024

<ipython-input-11-0074e4bdeffe>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2") # 784 -1024
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models 

In [12]:
vector1 = embedding_model.embed_query(pages_char_split[3].page_content)
vector2 = embedding_model.embed_query(pages_char_split[5].page_content)
vector3 = embedding_model.embed_query(pages_char_split[18].page_content)

In [13]:
np.dot(vector1, vector2) , np.dot(vector1, vector2)

(0.623600434636373, 0.623600434636373)

In [14]:
vectorstore = Chroma.from_documents(documents = pages_char_split,
                                    embedding = embedding_model,
                                    persist_directory = "/content/drive/MyDrive/UntitledFolde1")

In [15]:
# ADD Document to vectorstore
added_document = Document(page_content='Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis',
                          metadata={'Course Title': 'Introduction to Data and Data Science',
                                    'Lecture Title': 'Analysis vs Analytics'})

In [16]:
vectorstore.add_documents([added_document])

['a846b400-8139-42fe-979f-c8654aa89fbd']

In [17]:
  question = "What programming Language do data scientists use?"

In [18]:
# use chorma similtry
retrieved_docs = vectorstore.similarity_search(query=question, k=5)

In [19]:
retrieved_docs

[Document(metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}, page_content='Thus, we need a lot of computational power, and we can expect people to use the languages similar to those in the big data column. Apart from R, Python, and MATLAB, other, faster languages are used like Java, JavaScript, C, C++, and Scala. Cool. What we said may be wonderful, but that’s not all! By using one or more programming languages, people create application software or, as they are sometimes called, software solutions, that are adjusted for specific business needs'),
 Document(metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}, page_content='Thus, we need a lot of computational power, and we can expect people to use the languages similar to those in the big data column. 

In [ ]:
for i in retrieved_docs:
  print(f"Page Content : {i.page_content}\n-------\nLecture Title: {i.metadata['Lecture Title']}\n")

Page Content : Thus, we need a lot of computational power, and we can expect people to use the languages similar to those in the big data column. Apart from R, Python, and MATLAB, other, faster languages are used like Java, JavaScript, C, C++, and Scala. Cool. What we said may be wonderful, but that’s not all! By using one or more programming languages, people create application software or, as they are sometimes called, software solutions, that are adjusted for specific business needs
-------
Lecture Title: Programming Languages & Software Employed in Data Science - All the Tools You Need

Page Content : As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. They can solve a wide variety of

In [20]:
question2 = "what software  do data scientest use?"

retrive2 = vectorstore.similarity_search(
    query=question,
    k=3
)

for i in retrive2:
  print(f"Page Content : {i.page_content}\n______\n Lecture Title: {i.metadata['Lecture Title']}\n")

Page Content : Thus, we need a lot of computational power, and we can expect people to use the languages similar to those in the big data column. Apart from R, Python, and MATLAB, other, faster languages are used like Java, JavaScript, C, C++, and Scala. Cool. What we said may be wonderful, but that’s not all! By using one or more programming languages, people create application software or, as they are sometimes called, software solutions, that are adjusted for specific business needs
______
 Lecture Title: Programming Languages & Software Employed in Data Science - All the Tools You Need

Page Content : Thus, we need a lot of computational power, and we can expect people to use the languages similar to those in the big data column. Apart from R, Python, and MATLAB, other, faster languages are used like Java, JavaScript, C, C++, and Scala. Cool. What we said may be wonderful, but that’s not all! By using one or more programming languages, people create application software or, as they

In [21]:
# Problem in retrival is a duplicated
# The marginal reverenve
retrive3 = vectorstore.max_marginal_relevance_search(query=question2, k=3, lambda_mult=0.1)

for i in retrive3 :
  print(f"Page Contnet : {i.page_content}\n----\n Lecture Title :{i.metadata['Lecture Title']}\n")

Page Contnet : As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. They can solve a wide variety of business and data-related problems from beginning to the end
----
 Lecture Title :Programming Languages & Software Employed in Data Science - All the Tools You Need

Page Contnet : It’s actually a software framework which was designed to address the complexity of big data and its computational intensity. Most notably, Hadoop distributes the computational tasks on multiple computers which is basically the way to handle big data nowadays. Power BI, SaS, Qlik, and especially Tableau are top-notch examples of software designed for business intelligence visualizations
----
 Lecture Title :Progra

In [22]:
len(vectorstore.get()["documents"])

41

In [23]:
retrival = vectorstore.as_retriever(search_type='mmr', search_kwargs={"k": 3 , "lambda_mult":0.7})

In [ ]:
retrival

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f9a13265710>, search_type='mmr', search_kwargs={'k': 3, 'lambda_mult': 0.7})

In [24]:
question = "what software do data scientests use?"

In [25]:
retrived_docs = retrival.invoke(question)

In [26]:
retrived_docs

[Document(metadata={'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need', 'Course Title': 'Introduction to Data and Data Science'}, page_content='As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. They can solve a wide variety of business and data-related problems from beginning to the end'),
 Document(metadata={'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need', 'Course Title': 'Introduction to Data and Data Science'}, page_content='Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data scienc

In [ ]:
for i in retrived_docs:
  print(f"Page Content: {i.page_content}\n------\n Lecture Title:{i.metadata['Lecture Title']}")

Page Content: As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. They can solve a wide variety of business and data-related problems from beginning to the end
------
 Lecture Title:Programming Languages & Software Employed in Data Science - All the Tools You Need
Page Content: Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data science. Thank you for watching!
------
 Lecture Title:Programming Languages & Software Employed in Data Science - All the Tools You Need
Page Content: It’s actually a software framework which was designed to address the complexity of big data and its computational 

# Generation

In [27]:
api = os.environ.get("OPEN_AI_APIKEY")

In [28]:
TEMPLATE = """
Answer the following question:
{question}

To answer the question , use only the following context :
{context}

At the end of response, specify the name of lecture this context is taken from in the format:
Resources : "Lecture Title"
where "Lecture Title" should be substituted with the title of all resource lectures.
"""

prompt_templete = PromptTemplate.from_template(TEMPLATE)

In [29]:
chat = ChatOpenAI(model_name="gpt-4o-mini", model_kwargs={"seed": 365},
                  max_tokens=250, api_key=api)

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [30]:
question = "What software do data scientists use?"


In [31]:
chain = {"context": retrival,
         "question": RunnablePassthrough()} | prompt_templete |chat | StrOutputParser()

In [32]:
chain.invoke(question)

'Data scientists commonly use R and Python, which are the two most popular tools in the field. These programming languages have the advantage of being able to manipulate data and integrate with multiple data and data science software platforms. They are not limited to just mathematical and statistical computations; rather, they are adaptable and can address a wide variety of business and data-related problems from beginning to end.\n\nResources: "Programming Languages & Software Employed in Data Science - All the Tools You Need"'

In [33]:
print("""Data scientists commonly use R and Python, which are the two most popular tools in the field. These programming languages have the advantage of being able to manipulate data and integrate with multiple data science software platforms. They are adaptable and can address a wide variety of business and data-related problems from start to finish.\n\nResources: "Programming Languages & Software Employed in Data Science - All the Tools You Need"'""")

Data scientists commonly use R and Python, which are the two most popular tools in the field. These programming languages have the advantage of being able to manipulate data and integrate with multiple data science software platforms. They are adaptable and can address a wide variety of business and data-related problems from start to finish.

Resources: "Programming Languages & Software Employed in Data Science - All the Tools You Need"'
